In [67]:
import pandas as pd 
import requests 
from plyer import notification
from datetime import datetime
import notificacao
import sqlite3
import re


# Extraindo a tabela banks da API
url = "https://brasilapi.com.br/api/banks/v1"

bancos = requests.get(url)

data = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

if bancos.status_code == 200:
    bancos_json = bancos.json()

# Transformando as tabelas em um DataFrame
bancos_full = pd.DataFrame(bancos_json)

# Conectando com banco de dados e salvando os dados brutos
bancos_connect = sqlite3.connect('Projeto Final.db')
bancos_full.to_sql('Bancos Bruto', bancos_connect, if_exists='replace', index=False )

# Alterando o nome das colunas
bancos_full_renomeado = bancos_full.set_axis(['ISPB', 'Nome', 'Código', 'Nome Completo'], axis=1)

# Alterando a coluna Nome Completo para deixar todos nomes em maiusculo
bancos_full_renomeado['Nome Completo']=bancos_full_renomeado['Nome Completo'].str.upper()

notificacao.alerta(bancos, data, url)

#Tratamento de Missing Values
bancos_full_tratado = (bancos_full_renomeado.dropna())

# Ordenação pela coluna Código
bancos_full_tratado.sort_values('Código')

# Salvando os dados tratados no banco de dados e fechando a conexão
bancos_full_tratado.to_sql('Bancos Tratado', bancos_connect, if_exists='replace', index=False )
bancos_connect.close()


bancos_full_tratado


,ISPB,Nome,Código,Nome Completo
0,00000000,BCO DO BRASIL S.A.,1.0,BANCO DO BRASIL S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.
4,00122327,SANTINVEST S.A. - CFI,539.0,"SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INV..."
5,00204963,CCR SEARA,430.0,COOPERATIVA DE CREDITO RURAL SEARA - CREDISEARA
6,00250699,AGK CC S.A.,272.0,AGK CORRETORA DE CAMBIO S.A.
...,...,...,...,...
352,92856905,ADVANCED CC LTDA,117.0,ADVANCED CORRETORA DE CÂMBIO LTDA
353,92874270,BCO DIGIMAIS S.A.,654.0,BANCO DIGIMAIS S.A.
354,92875780,WARREN CVMC LTDA,371.0,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...
355,92894922,BANCO ORIGINAL,212.0,BANCO ORIGINAL S.A.


In [140]:
# Extraindo a tabela corretoras da API
url = "https://brasilapi.com.br/api/cvm/corretoras/v1"

corretoras = requests.get(url)

data = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

if corretoras.status_code == 200:
    corretoras_json = corretoras.json()

# Transformando as tabelas em um DataFrame
corretoras_full = pd.DataFrame(corretoras_json)

# Conectando com banco de dados e salvando os dados brutos
corretoras_connect = sqlite3.connect('Projeto Final.db')
corretoras_full.to_sql('Corretoras Bruto', corretoras_connect, if_exists='replace', index=False )

# Alterando o nome das colunas
corretoras_full_renomeado = corretoras_full.set_axis(['CNPJ', 'Tipo', 'Razão Social', 'Nome Comercial', 'Status', 'E-Mail', 'Telefone',
                          'CEP', 'Pais', 'UF', 'Municipio', 'Bairro', 'Complemento', 'Logradouro', 'Data Patrimonio', 
                          'Valor Patrimonio', 'Codigo CVM', 'Data Inicio', 'Data Registro'], axis=1)


notificacao.alerta(corretoras, data, url)

# Formatacao da coluna valor patrimonio

def formatar(valor):
    return "R${:,.2f}".format(valor)

corretoras_full_renomeado['Valor Patrimonio'] = pd.to_numeric(corretoras_full_renomeado['Valor Patrimonio'])
corretoras_full_renomeado['Valor Patrimonio'] = corretoras_full_renomeado['Valor Patrimonio'].apply(formatar)

# Formatacao das colunas com datas

corretoras_full_renomeado['Data Registro'] = pd.to_datetime(corretoras_full_renomeado['Data Registro'])
corretoras_full_renomeado['Data Registro'] = corretoras_full_renomeado['Data Registro'].dt.strftime('%d-%m-%Y')


# Exclusão de colunas irrelevantes para a análise. 

corretoras_full_colunas = corretoras_full_renomeado.drop(['Tipo','E-Mail', 'CEP', 'Complemento', 'Logradouro','Data Inicio','Data Patrimonio'], axis=1)

# Ajustando o Formato do CNPJ
padrao = r'(\d{2})(\d{3})(\d{3})(\d{4})(\d{2})'
corretoras_full_colunas['CNPJ'] = corretoras_full_colunas['CNPJ'].str.replace(padrao, r'\1.\2.\3/\4-\5', regex = True)

# Ajustando Formato do Telefone
filtro_tel =  corretoras_full_colunas['Telefone'].str.len() < 8 
corretoras_full_colunas.loc[filtro_tel, 'Telefone'] = '3'+ corretoras_full_colunas.loc[filtro_tel, 'Telefone']
padrao_tel = r'(\d{4})(\d{4})'
corretoras_full_colunas['Telefone'] = corretoras_full_colunas['Telefone'].str.replace(padrao_tel, r'\1-\2', regex=True)

# Excluindo corretoras que não possuem Telefone na base de dados
filtro_sem_tel =corretoras_full_colunas[ corretoras_full_colunas['Telefone'].str.len() == 1].index 
corretoras_full_colunas = corretoras_full_colunas.drop(filtro_sem_tel)

#Excluindo Corretoras que não possuem Pais na base de dados
filtro_pais = corretoras_full_colunas[corretoras_full_colunas['Pais'].str.len() == 0].index
corretoras_full_colunas = corretoras_full_colunas.drop(filtro_pais)

#Excluindo Corretoras que não possuem Municipio na base de dados
filtro_municipio = corretoras_full_colunas[corretoras_full_colunas['Municipio'].str.len() == 0].index
corretoras_full_colunas = corretoras_full_colunas.drop(filtro_municipio)

#Excluindo Corretoras que não possuem UF na base de dados
filtro_UF = corretoras_full_colunas[corretoras_full_colunas['UF'].str.len() == 0].index
corretoras_full_colunas = corretoras_full_colunas.drop(filtro_UF)

#Excluindo Corretoras que não possuem Bairro na base de dados
filtro_bairro = corretoras_full_colunas[corretoras_full_colunas['Bairro'].str.len() == 0].index
corretoras_full_colunas = corretoras_full_colunas.drop(filtro_bairro)

# Filtrando as corretoras que estão em funcionamento
corretoras_ativas = corretoras_full_colunas.loc[corretoras_full_colunas['Status'] =='EM FUNCIONAMENTO NORMAL']

# Filtrando as corretoras que estão canceladas
corretoras_canceladas = corretoras_full_colunas.loc[corretoras_full_colunas['Status'] =='CANCELADA']

# Salvando os dados tratados no banco de dados e fechando a conexão
corretoras_full_colunas.to_sql('Corretoras Tratado', corretoras_connect, if_exists='replace', index=False )
corretoras_ativas.to_sql('Corretoras Tratado', corretoras_connect, if_exists='replace', index=False )
corretoras_canceladas.to_sql('Corretoras Tratado', corretoras_connect, if_exists='replace', index=False )
bancos_connect.close()

corretoras_full_colunas


Corretoras Estado
UF Municipio                               
AM MANAUS                                 3
BA SALVADOR                               5
CE FORTALEZA                              7
DF BRASÍLIA                               1
ES VITÓRIA                                2
MG BELO HORIZONTE                         8
   POÇOS DE CALDAS                        1
PA BELÉM                                  1
PE RECIFE                                 9
PR CURITIBA                              13
RJ RIO DE JANEIRO                        48
RN NATAL                                  1
RS PORTO ALEGRE                           7
SC JOINVILLE                              1
SP BELO HORIZONTE                         1
   SANTOS                                 5
   SÃO BERNARDO DO CAMPO                  1
   SÃO PAULO                             85

In [ ]:

# Extraindo a tabela PIX da API
url = 'https://brasilapi.com.br/api/pix/v1/participants'

pix = requests.get(url)

data = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

if pix.status_code == 200:
    pix_json = pix.json()

# Transformando as tabelas em um DataFrame
pix_full = pd.DataFrame(pix_json)

# Conectando com banco de dados e salvando os dados brutos
pix_connect = sqlite3.connect('Projeto Final.db')
pix_full.to_sql('Pix Bruto', pix_connect, if_exists='replace', index=False )

# Alterando o nome das colunas
pix_full_renomeado = pix_full.set_axis(['ISPB', 'Nome', 'Nome Reduzido', 'Modalidade', 'Tipo', 'Inicio'], axis=1)

notificacao.alerta(pix, data, url)

# Tratamento da coluna Inicio que contem data e hora
pix_full_renomeado['Inicio'] = pd.to_datetime(pix_full_renomeado['Inicio'])
pix_full_renomeado['Inicio'] = pix_full_renomeado['Inicio'].dt.strftime('%d-%m-%Y %H:%M')

# Salvando os dados tratados no banco de dados e fechando a conexão
pix_full_renomeado.to_sql('Pix Tratado', pix_connect, if_exists='replace', index=False )
pix_connect.close()


pix_full_renomeado
